### Outline:

0. Load libraries and custom functions.
1. Load data.
2. Preliminary data analysis: explore features and a target, delete unneeded features, create new features.
3. Train-test split.
4. Missing values. In some cases it may be useful to explore skew and perform log-transform before imputing missing values.
5. Feature engineering. Transform skewed variables, do OHC and scaling.
6. Fit models.
7. Evaluate models.
8. Feature importance, error analysis. Based on the results, go to 2. and iterate.
9. Make predictions.

To do: 
- try fitting separate XGBoost for each feature.



In [12]:
import numpy as np
import pandas as pd
import os, time, warnings, random
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict, GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 20)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore')

def draw_histograms(df, variables, n_rows, n_cols):
    # stolen from https://stackoverflow.com/questions/29530355/plotting-multiple-histograms-in-grid
    fig=plt.figure()
    for i, var_name in enumerate(variables):
        ax=fig.add_subplot(n_rows,n_cols,i+1)
        df[var_name].hist(bins=10,ax=ax)
        ax.set_title(var_name+" Distribution")
    fig.tight_layout()  
    plt.show()


def fillna_mp_i1(df_train, df_test, df_pred, num_features, cat_features, num_fill='median', cat_fill='mode'):
    """This function speeds up filling missing values for 3 main datasets using different imputation methods.
    Later may replace it with some subclass.
    Example: fillna_mp_i1(X_train, X_test, X_pred, num_cols, cat_cols)"""
    # set df_pred to None if it does not exist
    if not ((cat_fill=='mode') and (num_fill=='median')):
        print ('Imputation method not Implemented yet!')
        return None
    
    df_train[num_features] = df_train[num_features].fillna(value=df_train[num_features].median())
    df_test[num_features] = df_test[num_features].fillna(value=df_train[num_features].median())
    df_train[cat_features] = df_train[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_test[cat_features] = df_test[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    if (df_pred is not None):
        df_pred[num_features] = df_pred[num_features].fillna(value=df_train[num_features].median())
        df_pred[cat_features] = df_pred[cat_features].fillna(value=df_train[cat_features].mode().iloc[0])
    df_train[num_features+cat_features].count
    
    all_good = (
    (np.prod(df_train[num_features+cat_features].shape)==df_train[num_features+cat_features].count().sum()) and 
    (np.prod(df_test[num_features+cat_features].shape) == df_test[num_features+cat_features].count().sum()) and 
    (np.prod(df_pred[num_features+cat_features].shape) == df_pred[num_features+cat_features].count().sum()))
    if (all_good):
        print('Missing values imputed successfully')
    else:
        print('There are still some missing values...')
    
def add_misDummy_mp_i1(df_train, df_test, df_pred, features):
    """This function creates new dummy columns for missing features.
    Example: add_misDummy_mp_i1(X_train, X_test, X_pred, ['Age'])"""
    # set df_pred to None if it does not exist
    for feature_name in features:
        misColName = 'mis'+feature_name
        df_train.loc[df_train[feature_name].isnull(), misColName]=1
        df_train.loc[df_train[feature_name].notnull(), misColName]=0
        df_test.loc[df_test[feature_name].isnull(), misColName]=1
        df_test.loc[df_test[feature_name].notnull(), misColName]=0
        if (df_pred is not None):
            df_pred.loc[df_pred[feature_name].isnull(), misColName]=1
            df_pred.loc[df_pred[feature_name].notnull(), misColName]=0
   

def discretize_mp_i1(df_train, df_test, df_pred, feature, ntiles, delete_feature=False):
    """This function divides a continuous feature into quantile groups.
    Example: discretize_mp_i1(X_train, X_test, X_pred, 'Age', 15)"""
    # set df_pred to None if it does not exist
    _,bin = pd.qcut(df_train[feature], ntiles, retbins = True, labels = False, duplicates = 'drop')
    df_train[feature+'Ntile'] = pd.cut(df_train[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    df_test[feature+'Ntile'] = pd.cut(df_test[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (df_pred is not None):
        df_pred[feature+'Ntile'] = pd.cut(df_pred[feature], labels=False, duplicates = 'drop', bins = bin ,include_lowest = True)
    if (delete_feature==True):
        df_train.drop(columns=[feature], inplace=True)
        df_test.drop(columns=[feature], inplace=True)
        df_pred.drop(columns=[feature], inplace=True)
    print('Discretized ',feature, ' into ', len(bin)-1, ' bins')


def log_transformer_mp_i1(df_train, df_test, df_pred, feature_subset=False, min_skew=3):
    """This function divides a continuous feature into quantile groups.
    Example: log_transformer_mp_i1(X_train, X_test, X_pred, feature_subset=num_cols)"""
    # set df_pred to None if it does not exist
    if (feature_subset==False):
        features_totransform = df_train.columns
    else:
        features_totransform = feature_subset.copy()
    skewed_vars = list(df_train.skew()[abs(df_train.skew())>min_skew].index)
    for col in list(set(skewed_vars)&set(features_totransform)):
        df_train[col] = np.log1p(df_train[col])
        df_test[col] = np.log1p(df_test[col])
        if (df_pred is not None):
            df_pred[col] = np.log1p(df_pred[col])
    print('Skewed columns log-transformed: ', list(set(skewed_vars)&set(features_totransform)))
    
    
def add_dummyfeatures(df_train, df_test, df_pred, feature_dict):
    """This function adds dummy feature when some feature is equal to value, specified in a dictionary.
    Example: add_dummyfeatures(X_train, X_test, X_pred, {'RoomService':0, 'Spa':0, 'VRDeck':0, 'ShoppingMall':0})"""
    input_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    for i in range(len(list(feature_dict.items()))):
        feature,value = list(feature_dict.keys())[i], list(feature_dict.values())[i]
        df_train.loc[df_train[feature]==value,(str(feature)+str(value))]=1
        df_train.loc[df_train[feature]!=value,(str(feature)+str(value))]=0
        df_test.loc[df_test[feature]==value,(str(feature)+str(value))]=1
        df_test.loc[df_test[feature]!=value,(str(feature)+str(value))]=0
        df_pred.loc[df_pred[feature]==value,(str(feature)+str(value))]=1
        df_pred.loc[df_pred[feature]!=value,(str(feature)+str(value))]=0
    output_dimensions = np.array([df_train.shape[1], df_test.shape[1], df_pred.shape[1]])
    print(output_dimensions-input_dimensions, ' variables created') 
    

# 1. Import data #

time0 = time.time()

path = '../input/santander-customer-transaction-prediction/train.csv'
df = pd.read_csv(path) 
df0 = df.copy()
#df = df.sample(50000)

#df.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)
pred=pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
pred0 = pred.copy()
#pred.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'],inplace=True)

print(df.shape, pred.shape)
print(df.target.mean())
# unbalanced responsed variable
df.head()

# 2. EDA #

# with all features unnamed normally-distributed features, there is not much EDA and feature engineering to do.
df.drop(columns = ['ID_code'], inplace= True)

# 3. Train-test split #

train_y = df[['target']]
train_x = df.drop(columns = ['target'])
X_pred = pred.copy()
X_pred.drop(columns = ['ID_code'], inplace= True)
#train_x = train_x[features_i1]
#X_pred = X_pred[features_i1]

X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.02, random_state=10)
print(X_train.shape, X_test.shape, y_train.shape, X_pred.shape)

X_train.count().sum() == np.prod(X_train.shape)
# no missing values, good.

# 5. feature engineering #

ss = StandardScaler()

for col in X_train.columns:
    X_train[[col]] = ss.fit_transform(X_train[[col]])
    X_test[[col]] = ss.transform(X_test[[col]])
    X_pred[[col]] = ss.transform(X_pred[[col]])

#X_test.iloc[:,:30].describe()
random.seed(1)
fewfeatures = random.sample(list(X_train.columns),5)

# 5.1 try PCA 

#pca = PCA(n_components=100)
#X_train = pca.fit_transform(X_train)
#X_test = pca.transform(X_test)


# 6. Fit models #

time1 = time.time()
lr = LogisticRegression()
param_grid = {'C':[0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1]}
lrm = GridSearchCV(lr, param_grid, cv=4, scoring='roc_auc')
lrm.fit(X_train, y_train)
#print('Logistic f1 ', lrm.best_params_, lrm.best_score_, f1_score(y_train, lrm.predict(X_train)), time.time()-time1)
print('Logistic roc_auc ', lrm.best_params_, lrm.best_score_, 
      roc_auc_score(y_train, lrm.predict_proba(X_train)[:,1]), time.time()-time1)


#time1 = time.time()
#knn = KNeighborsClassifier(n_jobs=-1)
#param_grid = dict(n_neighbors=range(10, 41, 10))
#knnm = GridSearchCV(knn, param_grid, cv=2)
#knnm.fit(X_train[fewfeatures], y_train)
#print('KNN ', knnm.best_params_, f1_score(y_train, knnm.predict(X_train[fewfeatures])), time.time()-time1)

#time1 = time.time()
#rf = RandomForestClassifier(n_jobs=-1)
#param_grid = {'n_estimators':[100], 'max_depth':[4,5,6], 'max_features':[10]}
#rfm = GridSearchCV(rf, param_grid, cv=2, scoring = 'f1')
#rfm.fit(X_train, y_train)
#print('RF ', rfm.best_params_, rfm.best_score_, f1_score(y_train, rfm.predict(X_train)), time.time()-time1)

time1 = time.time()
param_grid_nb = {
    'var_smoothing': np.logspace(-3,-7, num=17)
}
nb = GaussianNB()
nbm = GridSearchCV(nb, param_grid_nb, cv=4, scoring='roc_auc')
nbm.fit(X_train, y_train)
print('NB ', nbm.best_params_, nbm.best_score_, 
      roc_auc_score(y_train, nbm.predict_proba(X_train)[:,1]), time.time()-time1)


time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[8000], 'max_depth':[3], 'eta':[0.02],
'subsample':[0.5],'colsample_bytree':[0.005]}
xgbm = GridSearchCV(xgb, param_grid, cv=4, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
      roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)

# 7. accuracy #

print('Out of Sample:')
print('Logistic ', roc_auc_score(y_test, lrm.predict_proba(X_test)[:,1]))
#print('SVM ', accuracy_score(y_test, svmm.predict(X_test)))
#print('KNN ', accuracy_score(y_test, knnm.predict(X_test[fewfeatures])))
print('Bayes ', roc_auc_score(y_test, nbm.predict_proba(X_test)[:,1]))
#print('RF ', f1_score(y_test, rfm.predict(X_test)))
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

# VotingClassifier:

estimator1 = []
estimator1.append(('LR', LogisticRegression(C=0.001)))
estimator1.append(('NB1', GaussianNB(var_smoothing = 1e-6)))
#estimator1.append(('NB2', GaussianNB(var_smoothing = 1e-7)))
#estimator.append(('RF', RandomForestClassifier(max_depth=4, max_features=10, n_estimators=100)))
estimator1.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1,
                                       eta=0.02, max_depth=3, n_estimators=8000, 
                                       subsample=0.5, colsample_bytree=0.005)))
vot_soft = VotingClassifier(estimators = estimator1, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator1, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifiers3 in sample', roc_auc_score(y_train, vot_soft.predict_proba(X_train)[:,1]), f1_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifiers3 out of sample', roc_auc_score(y_test, vot_soft.predict_proba(X_test)[:,1]), f1_score(y_test, vot_hard.predict(X_test)))

estimator3 = []
#estimator3.append(('LR', LogisticRegression(C=0.001)))
estimator3.append(('NB1', GaussianNB(var_smoothing = 1e-6)))
#estimator1.append(('NB2', GaussianNB(var_smoothing = 1e-7)))
#estimator.append(('RF', RandomForestClassifier(max_depth=4, max_features=10, n_estimators=100)))
estimator3.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1,
                                       eta=0.02, max_depth=3, n_estimators=8000, 
                                       subsample=0.5, colsample_bytree=0.005)))
vot_soft = VotingClassifier(estimators = estimator3, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator3, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifiers3 in sample', roc_auc_score(y_train, vot_soft.predict_proba(X_train)[:,1]), f1_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifiers3 out of sample', roc_auc_score(y_test, vot_soft.predict_proba(X_test)[:,1]), f1_score(y_test, vot_hard.predict(X_test)))
print('Total time ', time.time()-time0)

(200000, 202) (200000, 201)
0.10049
(196000, 200) (4000, 200) (196000, 1) (200000, 200)
Logistic roc_auc  {'C': 0.0001} 0.8592985828598789 0.8608310903421739 25.896098375320435
NB  {'var_smoothing': 3.162277660168379e-07} 0.8880007953994473 0.8896255974500569 49.0909423828125
XGB  {'colsample_bytree': 0.005, 'eta': 0.02, 'max_depth': 3, 'n_estimators': 8000, 'subsample': 0.5} 0.8975434400723943 0.918000991580137 181.88582372665405
Out of Sample:
Logistic  0.8790826625674636
Bayes  0.9059843858656327
XGB  0.9141771610204088
VotingClassifiers3 in sample 0.8993361964337245 0.4099182273806383
VotingClassifiers3 out of sample 0.9089518033959194 0.43333333333333335
VotingClassifiers3 in sample 0.9073576197427422 0.3162098537077518
VotingClassifiers3 out of sample 0.9136872837431129 0.2929936305732484
Total time  482.79882979393005


In [ ]:
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1,
                   n_estimators=200, max_depth=2, eta=0.3, subsample = 0.4, colsample_bytree=0.3)
xgbm.fit(X_train, y_train)
print('XGB ', roc_auc_score(y_test, xgbm.predict(X_test)))

# VotingClassifier:

estimator1 = []
estimator1.append(('LR', LogisticRegression(C=0.002)))
estimator1.append(('NB1', GaussianNB(var_smoothing = 1e-6)))
#estimator1.append(('NB2', GaussianNB(var_smoothing = 1e-7)))
#estimator.append(('RF', RandomForestClassifier(max_depth=4, max_features=10, n_estimators=100)))
estimator1.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1,
                                       eta=0.3, max_depth=2, n_estimators=200, 
                                       subsample=0.4, colsample_bytree=0.3)))
vot_soft = VotingClassifier(estimators = estimator1, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator1, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifiers3 in sample', roc_auc_score(y_train, vot_soft.predict_proba(X_train)[:,1]), f1_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifiers3 out of sample', roc_auc_score(y_test, vot_soft.predict_proba(X_test)[:,1]), f1_score(y_test, vot_hard.predict(X_test)))

estimator2 = []
estimator2.append(('LR', LogisticRegression(C=0.002)))
estimator2.append(('NB1', GaussianNB(var_smoothing = 1e-5)))
estimator2.append(('NB2', GaussianNB(var_smoothing = 1e-8)))
#estimator.append(('RF', RandomForestClassifier(max_depth=4, max_features=10, n_estimators=100)))
estimator2.append(('XGB', XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1,
                                       eta=0.3, max_depth=2, n_estimators=200, 
                                       subsample=0.4, colsample_bytree=0.3)))
vot_soft = VotingClassifier(estimators = estimator2, voting ='soft')
vot_soft.fit(X_train, y_train)
vot_hard = VotingClassifier(estimators = estimator2, voting ='hard')
vot_hard.fit(X_train, y_train)
print('VotingClassifiers3 in sample', roc_auc_score(y_train, vot_soft.predict_proba(X_train)[:,1]), f1_score(y_train, vot_hard.predict(X_train)))
print('VotingClassifiers3 out of sample', roc_auc_score(y_test, vot_soft.predict_proba(X_test)[:,1]), f1_score(y_test, vot_hard.predict(X_test)))
print('total time is ', time.time()-time0)

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[200], 'max_depth':[2], 'eta':[0.3],
'subsample':[0.5],'colsample_bytree':[0.1]}
xgbm = GridSearchCV(xgb, param_grid, cv=2, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[2000, 3000], 'max_depth':[3,4,5,6,8], 'eta':[0.01, 0.03, 0.05, 0.1],
'subsample':[0.4,0.5,0.6],'colsample_bytree':[0.005]}
xgbm = RandomizedSearchCV(xgb, param_grid, random_state=0, cv=2, n_iter=10, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[3000], 'max_depth':[3,4,5,6], 'eta':[0.05, 0.1, 0.15, 0.2],
'subsample':[0.5,0.6],'colsample_bytree':[0.005]}
xgbm = RandomizedSearchCV(xgb, param_grid, random_state=0, cv=2, n_iter=10, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[3000], 'max_depth':[3,4], 'eta':[0.05, 0.1, 0.15, 0.2],
'subsample':[0.5,0.6],'colsample_bytree':[0.005]}
xgbm = RandomizedSearchCV(xgb, param_grid, random_state=0, cv=2, n_iter=8, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

In [21]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[5000], 'max_depth':[3], 'eta':[0.06, 0.08, 0.1],
'subsample':[0.6],'colsample_bytree':[0.005]}
xgbm = GridSearchCV(xgb, param_grid, cv=4, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB ', xgbm.best_params_, xgbm.best_score_, 
      roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

XGB  {'colsample_bytree': 0.005, 'eta': 0.08, 'max_depth': 3, 'n_estimators': 3000, 'subsample': 0.6} 0.8954880884107757 0.9193106050903986 193.8829221725464
XGB  0.8882499359150693


In [8]:
time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[5000], 'max_depth':[3], 'eta':[0.03, 0.04, 0.05],
'subsample':[0.6],'colsample_bytree':[0.005]}
xgbm = GridSearchCV(xgb, param_grid, cv=4, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB 5000', xgbm.best_params_, xgbm.best_score_, 
      roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

time1 = time.time()
xgb = XGBClassifier(tree_method='gpu_hist', gpu_id=0, min_child_weight=50, n_jobs=-1)
param_grid = {'n_estimators':[8000], 'max_depth':[3], 'eta':[0.015, 0.02, 0.025],
'subsample':[0.5],'colsample_bytree':[0.005]}
xgbm = GridSearchCV(xgb, param_grid, cv=4, scoring='roc_auc')
xgbm.fit(X_train, y_train)
print('XGB 8000', xgbm.best_params_, xgbm.best_score_, 
roc_auc_score(y_train, xgbm.predict_proba(X_train)[:,1]), time.time()-time1)
print('XGB ', roc_auc_score(y_test, xgbm.predict_proba(X_test)[:,1]))

XGB 5000 {'colsample_bytree': 0.005, 'eta': 0.03, 'max_depth': 3, 'n_estimators': 5000, 'subsample': 0.6} 0.8963482411450597 0.9167435137473001 285.34923791885376
XGB  0.9139258877617958
XGB 8000 {'colsample_bytree': 0.005, 'eta': 0.02, 'max_depth': 3, 'n_estimators': 8000, 'subsample': 0.5} 0.8975434400723943 0.918000991580137 447.38973736763
XGB  0.9141771610204088


In [11]:
# 8. feature importance #

results = permutation_importance(xgbm, X_test, y_test, scoring='f1', n_jobs=-1)
fi_lr = pd.DataFrame({'col':X_test.columns, 'FI':results.importances_mean})
fi_lr.sort_values('FI', ascending = False)[:20]

KeyboardInterrupt: 

In [ ]:
temp = fi_lr.sort_values('FI', ascending = False)

In [ ]:
features_i1 = list(temp.loc[temp.FI>0]['col'])
features_i1

In [ ]:
temp

In [14]:
# 9. predictions #

submission_df_vc = pd.DataFrame({'ID_code': pred0.ID_code, 'target': vot_soft.predict_proba(X_pred)[:,1]}, columns=['ID_code', 'target'])
submission_df_bt = pd.DataFrame({'ID_code': pred0.ID_code, 'target': xgbm.predict_proba(X_pred)[:,1]}, columns=['ID_code', 'target'])

#submission_df_bt.Transported = np.array([bool(x) for x in submission_df_bt.Transported])

submission_df_vc.to_csv('KP12_vc.csv',index=False)
#submission_df_svm.to_csv('KP11_svm.csv',index=False)
#submission_df_rf.to_csv('KP11_rf.csv',index=False)
submission_df_bt.to_csv('KP12_bt.csv',index=False)

os.chdir(r'/kaggle/working')

from IPython.display import FileLink
FileLink(r'KP12_vc.csv')

/kaggle/working/KP12_vc.csv

In [ ]:
X_pred

In [ ]:
df

In [ ]:
np.logspace(-3,-7, num=17)